In [1]:
!pip install yfinance
!pip install statsmodels==0.11.0

import os, contextlib
import pandas as pd
import yfinance as yf

if not os.path.exists("stocks data"):
    os.mkdir("stocks data")
    
url = "https://en.wikipedia.org/wiki/NASDAQ-100#Components"
html = pd.read_html(url, header=0)
series = html[4]["Ticker"]
symbols = series.to_list()

with open(os.devnull, "w") as devnull:
    with contextlib.redirect_stdout(devnull):
        for i, symbol in enumerate(symbols):
            data = yf.download(symbol, period="max")["Close"]
            data.to_csv("/content/stocks data/{}.csv".format(symbol))



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.1 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.7 MB 5.1 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.12.2
    Uninstalling statsmodels-0.12.2:
      Successfully uninstalled statsmodels-0.12.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
plotnine 0.8.0 requires statsmodels>=0.12.1, but you have statsmodels 0.11.0 which is incompatible.


In [2]:
!pip install sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=b7e06a27a6fe4e0b8e7512a2df774229216a7925afa060d205e849f0d6851256
  Stored in directory: /root/.cache/pip/wheels/42/56/cc/4a8bf86613aafd5b7f1b310477667c1fca5c51c3ae4124a003
Successfully built sklearn


In [3]:
import numpy as np
import pandas as pd
import matplotlib as plt
import yfinance as yf
import plotly.express as px
from datetime import datetime
from statsmodels.tsa.stattools import acf , pacf
from statsmodels.graphics.tsaplots import plot_acf
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

final_forecasts = {}
p = 2
q = 1
prediction_length = 5
d = {}
valid_size = 0.15

for symbol in symbols:
    stock = yf.download(symbol, period="MAX", Interval="1d", verbose=False)
    # closing_prices = stock.Close.dropna()
    # original_cp = closing_prices
    # assert closing_prices.isnull().sum() == 0
    # d_counter = 0
    # while not stationarity_test(closing_prices):
    #     closing_prices = closing_prices.diff(1).bfill()
    #     d_counter += 1
    # d[symbol] = d_counter

    df = stock
    valid_split_idx = int(df.shape[0] * (1-(valid_size)))

    train_df  = df[:valid_split_idx].copy()
    valid_df  = df[valid_split_idx:-5].copy()
    test_df   = df[-5:].copy()
    y_train = train_df['Close'][5:].copy()
    X_train = train_df.drop(['Close'], 1)[:-5]

    y_valid = valid_df['Close'].copy()
    X_valid = valid_df.drop(['Close'], 1)

    y_test  = test_df['Close'].copy()
    X_test  = test_df.drop(['Close'], 1)
    eval_set = [(X_train, y_train), (X_valid, y_valid)]

    # create dataset for lightgbm
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

    # specify your configurations as a dict
    params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
    }
    # train
    gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_eval,
                callbacks=[lgb.early_stopping(stopping_rounds=5)])
    # predict
    y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)[:5]

    # eval
    rmse_test = mean_squared_error(y_test, y_pred) ** 0.5
    print(f'The RMSE of prediction is: {rmse_test}')

    final_forecasts[symbol] = y_pred
















[*********************100%***********************]  1 of 1 completed
[1]	valid_0's l2: 3204.93	valid_0's l1: 56.6112
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's l2: 2878.78	valid_0's l1: 53.6532
[3]	valid_0's l2: 2585.14	valid_0's l1: 50.8432
[4]	valid_0's l2: 2320.81	valid_0's l1: 48.1736
[5]	valid_0's l2: 2082.9	valid_0's l1: 45.6376
[6]	valid_0's l2: 1873.34	valid_0's l1: 43.2808
[7]	valid_0's l2: 1682.34	valid_0's l1: 41.015
[8]	valid_0's l2: 1510.4	valid_0's l1: 38.8624


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


[9]	valid_0's l2: 1355.64	valid_0's l1: 36.8175
[10]	valid_0's l2: 1216.36	valid_0's l1: 34.8748
[11]	valid_0's l2: 1089.96	valid_0's l1: 33.0128
[12]	valid_0's l2: 980.778	valid_0's l1: 31.3157
[13]	valid_0's l2: 878.295	valid_0's l1: 29.6342
[14]	valid_0's l2: 786.174	valid_0's l1: 28.0369
[15]	valid_0's l2: 703.386	valid_0's l1: 26.5194
[16]	valid_0's l2: 628.537	valid_0's l1: 25.0685
[17]	valid_0's l2: 562.935	valid_0's l1: 23.7242
[18]	valid_0's l2: 503.964	valid_0's l1: 22.447
[19]	valid_0's l2: 449.578	valid_0's l1: 21.201
[20]	valid_0's l2: 400.788	valid_0's l1: 20.0173
Did not meet early stopping. Best iteration is:
[20]	valid_0's l2: 400.788	valid_0's l1: 20.0173
The RMSE of prediction is: 20.01969239397605
[*********************100%***********************]  1 of 1 completed
[1]	valid_0's l2: 69765.1	valid_0's l1: 264.035
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's l2: 66973.8	valid_0's l1: 258.695
[3]	valid_0's l2: 64454.1	valid_0's l1: 253.778

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


[*********************100%***********************]  1 of 1 completed
[1]	valid_0's l2: 44510	valid_0's l1: 210.95
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's l2: 43274.1	valid_0's l1: 208
[3]	valid_0's l2: 42113.1	valid_0's l1: 205.19
[4]	valid_0's l2: 41024.8	valid_0's l1: 202.521
[5]	valid_0's l2: 40020.9	valid_0's l1: 200.027
[6]	valid_0's l2: 39061.8	valid_0's l1: 197.615
[7]	valid_0's l2: 38153.8	valid_0's l1: 195.304
[8]	valid_0's l2: 37284.1	valid_0's l1: 193.065
[9]	valid_0's l2: 36501.9	valid_0's l1: 191.028


...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


[*********************100%***********************]  1 of 1 completed
[1]	valid_0's l2: 19826.2	valid_0's l1: 140.778
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's l2: 17950.1	valid_0's l1: 133.952
[3]	valid_0's l2: 16214.1	valid_0's l1: 127.307
[4]	valid_0's l2: 14668.6	valid_0's l1: 121.086
[5]	valid_0's l2: 13251.8	valid_0's l1: 115.086
[6]	valid_0's l2: 12007.5	valid_0's l1: 109.551
[7]	valid_0's l2: 10866.1	valid_0's l1: 104.211
[8]	valid_0's l2: 9850.85	valid_0's l1: 99.2231
[9]	valid_0's l2: 8934.36	valid_0's l1: 94.4952
[10]	valid_0's l2: 8102.32	valid_0's l1: 89.9857
[11]	valid_0's l2: 7355.49	valid_0's l1: 85.738
[12]	valid_0's l2: 6678.13	valid_0's l1: 81.695
[13]	valid_0's l2: 6072.16	valid_0's l1: 77.898
[14]	valid_0's l2: 5523.2	valid_0's l1: 74.2909
[15]	valid_0's l2: 5020.47	valid_0's l1: 70.8292
[16]	valid_0's l2: 4559.13	valid_0's l1: 67.4959
[17]	valid_0's l2: 4141.87	valid_0's l1: 64.333
[18]	valid_0's l2: 3764.36	valid_0's l1: 61.331
[19

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


[*********************100%***********************]  1 of 1 completed
[1]	valid_0's l2: 27.8293	valid_0's l1: 4.22791
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's l2: 26.5	valid_0's l1: 4.32217
[3]	valid_0's l2: 27.2241	valid_0's l1: 4.56492
[4]	valid_0's l2: 28.2153	valid_0's l1: 4.79553
[5]	valid_0's l2: 29.3544	valid_0's l1: 5.00217
[6]	valid_0's l2: 26.4812	valid_0's l1: 4.7903
Early stopping, best iteration is:
[1]	valid_0's l2: 27.8293	valid_0's l1: 4.22791
The RMSE of prediction is: 5.275352011020994


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


In [4]:
import numpy as np
np.save('forecasts_companies.npy', final_forecasts)
